At the moment, the script works with only one card set at a time.  Set the card set UUID:

In [79]:
SET_UUID = '8a3273ca-1ccd-4e07-913b-766fcc49fe6f'
#SET_UUID = '17b90ca1-7e3e-4891-99da-954a7833e88e'

Set whether you want to generate all cards from scratch (`True`) or to update only the cards, changed since the previous script run (`False`):

In [80]:
FROM_SCRATCH = True

Run the cells below:

In [117]:
import hashlib
import math
import os
import py7zr
import re
import requests
import shutil
import subprocess
import xlwings as xw
import xml.etree.ElementTree as ET
import yaml
import zipfile

from reportlab.lib.pagesizes import landscape, letter, A4
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader
from reportlab.pdfgen.canvas import Canvas

PROJECT_FOLDER = 'Frogmorton'
SHEET_NAME = 'setExcel'

CONFIGURATION_PATH = 'configuration.yaml'
IMAGES_BACK_PATH = 'imagesBack'
IMAGES_EONS_PATH = 'imagesEons'
IMAGES_RAW_PATH = os.path.join(PROJECT_FOLDER, 'imagesRaw')
IMAGES_ZIP_PATH = '{}/Export/'.format(os.path.split(PROJECT_FOLDER)[-1])
MACROS_PATH = 'macros.xlsm'
MACROS_COPY_PATH = 'macros_copy.xlsm'
OCTGN_ZIP_PATH = 'imagesOCTGN/a21af4e8-be4b-4cda-a6b6-534f9717391f/Sets'
OUTPUT_DB_PATH = os.path.join('Output', 'DB')
OUTPUT_MPC_PATH = os.path.join('Output', 'MakePlayingCards')
OUTPUT_OCTGN_PATH = os.path.join('Output', 'OCTGN')
OUTPUT_PDF_PATH = os.path.join('Output', 'PDF')
PROJECT_PATH = 'setGenerator.seproject'
SET_EONS_PATH = 'setEons'
TEMP_PATH = 'Temp'
XML_PATH = os.path.join(PROJECT_FOLDER, 'XML')

In [82]:
# Read configuration
with open(CONFIGURATION_PATH, 'r') as f_conf:
    conf = yaml.safe_load(f_conf)

In [83]:
# Download the cards spreadsheet from Google Drive
sheet_path = '{}.{}'.format(SHEET_NAME, conf['sheet_type'])
if conf['sheet_type'] == 'xlsm':
    url = 'https://drive.google.com/uc?export=download&id={}'.format(conf['sheet_gdid'])
else:
    url = 'https://docs.google.com/spreadsheets/d/{}/export?format=xlsx'.format(conf['sheet_gdid'])

with open(sheet_path, 'wb') as f_sheet:
    f_sheet.write(requests.get(url).content)

In [84]:
# Backup the previous setEons.xml
if os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml')):
    _ = shutil.move(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'),
                os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))

if FROM_SCRATCH and os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old')):
    os.remove(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))

In [85]:
# Copy Excel Sheet over to sheet that contains the macros, and run the macros.
# This will create two .xml files. One in the setEons folder called setEons.xml,
# which contains the card data needed by Strange Eons.
# The second will be placed in the Output/OCTGN folder, which should be copied over to your OCTGN folder.

card_data_range = 'A2:AU1001'

_ = shutil.copyfile(MACROS_PATH, MACROS_COPY_PATH)

set_name = None

app = xw.App(visible=False)  # if you want Excel to run in background
xlws = {}

xlwb1 = xw.Book(sheet_path)
try:
    xlws['ws1a'] = xlwb1.sheets['Sets']
    xlws['ws1b'] = xlwb1.sheets['Card Data']

    set_row = None
    for row in range(3, 102):
        if xlws['ws1a'].range((row, 1)).value == SET_UUID:
            set_row = row
            set_name = xlws['ws1a'].range((row, 2)).value
            break

    if set_row:
        
        xlwb2 = xw.Book(MACROS_COPY_PATH)
        try:
            xlws['ws2a'] = xlwb2.sheets['Sets']
            xlws['ws2b'] = xlwb2.sheets['Card Data']

            newVal = xlws['ws1a'].range('A{}:B{}'.format(set_row, set_row)).value  # get set id/name
            xlws['ws2a'].range('A3:B3').value = newVal  # update set id/name

            newVal = xlws['ws1b'].range(card_data_range).value  # get card data
            xlws['ws2b'].range(card_data_range).value = newVal  # update card data

            xlws['ws2b'].range(card_data_range).api.Sort(
                Key1=xlws['ws2b'].range('Set').api,
                Order1=xw.constants.SortOrder.xlAscending,
                Key2=xlws['ws2b'].range('CardNumber').api,
                Order2=xw.constants.SortOrder.xlAscending)

            # This macro will generate a folder under Output/OCTGN, which should be copied over
            # to OCTGN/GameDatabase/a21af4e8-be4b-4cda-a6b6-534f9717391f/Sets 
            macro1 = xlwb2.macro('SaveOCTGN')
            macro1()

            # This macro will generate a file called setEons.xml, which is used by the Strange Eons script.
            macro2 = xlwb2.macro('SaveXML')
            macro2()

            xlwb2.save()
        finally:
            xlwb2.close()
    else:
        print('Set {} not found in the spreadsheet'.format(SET_UUID))    
finally:
    xlwb1.close()

In [86]:
# Find properties with a given name
def find_properties(parent, name):
    return [p for p in parent if p.attrib.get('name') == name]

# Get new or existing property with a given name
def get_property(parent, name):
    properties = find_properties(parent, name)
    if properties:
        prop = properties[0]
    else:
        prop = ET.SubElement(parent, 'property')
        prop.set('name', name)

    return prop

# Delete old image files
for _, _, filenames in os.walk(IMAGES_RAW_PATH):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            os.remove(os.path.join(IMAGES_RAW_PATH, filename))

    break

# Parse artwork images
images = {}
for _, _, filenames in os.walk(conf['artwork_path']):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            card_id_side = '_'.join(filename.split('_')[:2])
            images[card_id_side] = filename

    break

# Update setEons.xml with artwork, artists and encounter set numbers
tree = ET.parse(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'))
root = tree.getroot()
encounter_sets = {}
encounter_cards = {}

for card in root[0]:
    card_type = find_properties(card, 'Type')[0].attrib['value']
    encounter_set = find_properties(card, 'Encounter Set')
    if card_type != 'Quest' and encounter_set:
        encounter_set = encounter_set[0].attrib['value']
        encounter_cards[card.attrib['id']] = encounter_set
        prop = get_property(card, 'Encounter Set Number')
        prop.set('value', str(encounter_sets.get(encounter_set, 0) + 1))
        quantity = int(find_properties(card, 'Quantity')[0].attrib['value'])
        encounter_sets[encounter_set] = encounter_sets.get(encounter_set, 0) + quantity

    filename = images.get('{}_{}'.format(card.attrib['id'], 'A'))
    if filename:
        prop = get_property(card, 'Artwork')
        prop.set('value', filename)
        prop = get_property(card, 'Artwork Size')
        prop.set('value', str(os.path.getsize(os.path.join(conf['artwork_path'], filename))))
        prop = get_property(card, 'Artwork Modified')
        prop.set('value', str(int(os.path.getmtime(os.path.join(conf['artwork_path'], filename)))))

        artist = find_properties(card, 'Artist')
        if not artist and '_Artist_' in filename:
            artist_value = '.'.join('_Artist_'.join(
                filename.split('_Artist_')[1:]).split('.')[:-1]).replace('_', ' ')
            prop = get_property(card, 'Artist')
            prop.set('value', artist_value)

    filename = images.get('{}_{}'.format(card.attrib['id'], 'B'))
    alternate = [a for a in card if a.attrib.get('type') == 'B']
    if filename and alternate:
        alternate = alternate[0]
        prop = get_property(alternate, 'Artwork')
        prop.set('value', filename)
        prop = get_property(alternate, 'Artwork Size')
        prop.set('value', str(os.path.getsize(os.path.join(conf['artwork_path'], filename))))
        prop = get_property(alternate, 'Artwork Modified')
        prop.set('value', str(int(os.path.getmtime(os.path.join(conf['artwork_path'], filename)))))

        artist = find_properties(alternate, 'Artist')
        if not artist and '_Artist_' in filename:
            artist_value = '.'.join('_Artist_'.join(
                filename.split('_Artist_')[1:]).split('.')[:-1]).replace('_', ' ')
            prop = get_property(alternate, 'Artist')
            prop.set('value', artist_value)

# Update setEons.xml with encounter set total and card hash
for card in root[0]:
    if card.attrib['id'] in encounter_cards:
        prop = get_property(card, 'Encounter Set Total')
        prop.set('value', str(encounter_sets[encounter_cards[card.attrib['id']]]))

    card_hash = hashlib.md5(re.sub('\n\s*', '', ET.tostring(card, encoding='unicode').strip()).encode()).hexdigest()
    card.set('hash', card_hash)

# Mark cards, which were not changed since the previous script run and copy artwork images into the project
old_hashes = {}
skip_ids = set()
if os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old')):
    tree_old = ET.parse(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))
    root_old = tree_old.getroot()
    for card in root_old[0]:
        old_hashes[card.attrib['id']] = card.attrib['hash']

for card in root[0]:
    if old_hashes.get(card.attrib['id']) == card.attrib['hash']:
        skip_ids.add(card.attrib['id'])
        card.set('skip', '1')
    else:
        filename = find_properties(card, 'Artwork')
        if filename:
            filename = filename[0].attrib['value']
            _ = shutil.copyfile(os.path.join(conf['artwork_path'], filename),
                                os.path.join(IMAGES_RAW_PATH, filename))

        alternate = [a for a in card if a.attrib.get('type') == 'B']
        if alternate:
            alternate = alternate[0]
            filename = find_properties(alternate, 'Artwork')
            if filename:
                filename = filename[0].attrib['value']
                _ = shutil.copyfile(os.path.join(conf['artwork_path'], filename),
                                    os.path.join(IMAGES_RAW_PATH, filename))

tree.write(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'))

# Copy setEons.xml into the project
_ = shutil.copyfile(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'),
                    os.path.join(XML_PATH, 'setEons.xml'))

In [87]:
# Create Strange Eons project archive
with zipfile.ZipFile(PROJECT_PATH, 'w') as zip_obj:
    for root, _, filenames in os.walk(PROJECT_FOLDER):
        for filename in filenames:
            zip_obj.write(os.path.join(root, filename))

### Manual Strange Eons Part:

1. Open `setGenerator.seproject` generated above.
2. Run `Script/makeCards` script by double clicking it.  This will take the templates, fill them with values from `setEons.xml` and export results in all the required formats.
3. Once completed, close Strange Eons.

Run the cells below:

In [103]:
# Create the folder if needed
def create_folder(folder):
    if not os.path.exists(folder):
        os.mkdir(folder)

# Clear the folder
def clear_folder(folder):
    for _, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.split('.')[-1] in ('jpg', 'png'):
                os.remove(os.path.join(folder, filename))

        break

# Delete folder images for outdated or modified cards
def clear_modified_images(folder, skip_ids):
    for _, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.split('.')[-1] in ('jpg', 'png'):
                card_id = filename[50:86]
                if card_id not in skip_ids:
                    os.remove(os.path.join(folder, filename))

        break

# Update filename found in the Strange Eons archive
def update_zip_filename(filename):
    output_filename = os.path.split(filename)[-1]
    output_filename = output_filename.encode('ascii', errors='replace').decode().replace('?', ' ')
    output_filename = re.sub('-2-1(?=\.(?:jpg|png)$)', '-2', output_filename)
    return output_filename

In [104]:
# Generate images for DB and OCTGN outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'jpg300NoBleed'))
                and f.split('.')[-1] == 'jpg']
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        with zip_obj.open(filename) as zip_file:
            with open(os.path.join(output_path, output_filename), 'wb') as output_file:
                _ = shutil.copyfileobj(zip_file, output_file)

In [105]:
# Generate images for PDF outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'png300PDF', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)
clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png300Bleed'))
                and f.split('.')[-1] == 'png']
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        if output_filename.endswith('-2.png'):
            with zip_obj.open(filename) as zip_file:
                with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                    _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-pdf-back-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png300NoBleed'))
                and f.split('.')[-1] == 'png']
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        if output_filename.endswith('-1.png'):
            with zip_obj.open(filename) as zip_file:
                with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                    _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-pdf-front-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

CompletedProcess(args='"C:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe" -i -b "(python-prepare-pdf-back-folder 1 \\"Temp\\" \\"imagesEons\\\\png300PDF\\\\8a3273ca-1ccd-4e07-913b-766fcc49fe6f\\")" -b "(gimp-quit 0)"', returncode=0, stdout=b'EEEEeEeek! 45 GeglBuffers leaked\n', stderr=b'batch command executed successfully\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe: GEGL-WARNING: (../gegl/buffer/gegl-tile-handler-cache.c:1076):gegl_tile_cache_destroy: runtime check failed: (g_queue_is_empty (&cache_queue))\nTo debug GeglBuffer leaks, set the environment variable GEGL_DEBUG to "buffer-alloc"\n')
CompletedProcess(args='"C:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe" -i -b "(python-prepare-pdf-front-folder 1 \\"Temp\\" \\"imagesEons\\\\png300PDF\\\\8a3273ca-1ccd-4e07-913b-766fcc49fe6f\\")" -b "(gimp-quit 0)"', returncode=0, stdout=b'EEEEeEeek! 926 GeglBuffers leaked\n', stderr=b'batch command executed successfully\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.

In [106]:
# Generate images for MakePlayingCards outputs
output_path = os.path.join(IMAGES_EONS_PATH, 'png800BleedMPC', SET_UUID)
create_folder(output_path)
clear_modified_images(output_path, skip_ids)
clear_folder(TEMP_PATH)

with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
    filelist = [f for f in zip_obj.namelist()
                if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, 'png800Bleed'))
                and f.split('.')[-1] == 'png']
    for filename in filelist:
        output_filename = update_zip_filename(filename)
        with zip_obj.open(filename) as zip_file:
            with open(os.path.join(TEMP_PATH, output_filename), 'wb') as output_file:
                _ = shutil.copyfileobj(zip_file, output_file)

cmd = '"{}" -i -b "(python-prepare-makeplayingcards-folder 1 \\"{}\\" \\"{}\\")" -b "(gimp-quit 0)"'.format(
    conf['gimp_console_path'],
    TEMP_PATH.replace('\\', '\\\\'),
    output_path.replace('\\', '\\\\'))
print(subprocess.run(cmd, capture_output=True, shell=True))

CompletedProcess(args='"C:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe" -i -b "(python-prepare-makeplayingcards-folder 1 \\"Temp\\" \\"imagesEons\\\\png800BleedMPC\\\\8a3273ca-1ccd-4e07-913b-766fcc49fe6f\\")" -b "(gimp-quit 0)"', returncode=0, stdout=b'EEEEeEeek! 989 GeglBuffers leaked\n', stderr=b'batch command executed successfully\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe: GEGL-WARNING: tile-backend-swap gap list had more than one element\n\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe: GEGL-WARNING: (../gegl/buffer/gegl-tile-backend-swap.c:1546):gegl_tile_backend_swap_cleanup: runtime check failed: (gap_list->start == 0 && gap_list->end == file_size)\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe: GEGL-WARNING: (../gegl/buffer/gegl-tile-handler-cache.c:1076):gegl_tile_cache_destroy: runtime check failed: (g_queue_is_empty (&cache_queue))\nTo debug GeglBuffer leaks, set the environment variable GEGL_DEBUG to "buffer-alloc"\n')


In [107]:
clear_folder(TEMP_PATH)

In [108]:
# Generate DB outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
output_path = os.path.join(OUTPUT_DB_PATH, set_name)
create_folder(output_path)
clear_folder(output_path)

known_filenames = set()
for _, _, filenames in os.walk(input_path):
    for filename in filenames:
        if filename.split('.')[-1] != 'jpg':
            continue

        db_filename = '{}-{}{}{}'.format(filename[:3], re.sub('-+$', '',filename[8:50]),
                                         re.sub('-1$', '', filename[86:88]), filename[88:])
        if db_filename not in known_filenames:
            known_filenames.add(db_filename)                
            _ = shutil.copyfile(os.path.join(input_path, filename),
                                os.path.join(output_path, db_filename))

    break

In [109]:
# Generate OCTGN outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'jpg300NoBleed', SET_UUID)
output_path = os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)
create_folder(output_path)

known_filenames = set()
pack_path = os.path.join(output_path, 'imagePack_{}.o8c'.format(SET_UUID))
with zipfile.ZipFile(pack_path, 'w', zipfile.ZIP_DEFLATED) as zip_obj:
    for _, _, filenames in os.walk(input_path):
        for filename in filenames:
            if filename.split('.')[-1] != 'jpg':
                continue

            parts = filename.split('.')
            parts[0] = re.sub('-1$', '', re.sub('-2$', '.B', parts[0]))
            octgn_filename = '.'.join(parts)[50:]
            if octgn_filename not in known_filenames:
                known_filenames.add(octgn_filename)
                zip_obj.write(os.path.join(input_path, filename),
                             '{}/{}/Cards/{}'.format(OCTGN_ZIP_PATH, SET_UUID, octgn_filename))

        break

In [110]:
# Generate PDF outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'png300PDF', SET_UUID)
output_path = os.path.join(OUTPUT_PDF_PATH, set_name)
create_folder(output_path)

pdf_images = {'player': [],
              'encounter': [],
              'custom': []}
for _, _, filenames in os.walk(input_path):
    for filename in filenames:
        parts = filename.split('-')
        if parts[-1] != '1.png':
            continue

        back_filepath = os.path.join(input_path, '{}-2.png'.format('-'.join(parts[:-1])))
        if os.path.exists(back_filepath):
            back_type = 'custom'
        else:
            if parts[2] == 'p':
                back_type = 'player'
                back_filepath = os.path.join(IMAGES_BACK_PATH, 'playerBackOfficial.png')
            elif parts[2] == 'e':
                back_type = 'encounter'
                back_filepath = os.path.join(IMAGES_BACK_PATH, 'encounterBackOfficial.png')
            else:
                print('Missing card back for {}, removing the file from PDF'.format(filename))
                continue

        copies = 3 if parts[1] == 'p' else 1
        for _ in range(copies):
            pdf_images[back_type].append((os.path.join(input_path, filename), back_filepath))
                
    break

cards_on_page = 6
pages_raw = []
for key in pdf_images:
    pages_raw.extend([(pdf_images[key][i * cards_on_page:(i + 1) * cards_on_page] +
                       [None] * cards_on_page)[:cards_on_page]
                      for i in range(math.ceil(len(pdf_images[key]) / cards_on_page))])

pages = []
for page in pages_raw:
    front_page = [i and i[0] or None for i in page]
    back_page = [i and i[1] or None for i in page]
    back_page = [back_page[2], back_page[1], back_page[0],
                 back_page[5], back_page[4], back_page[3]]
    pages.extend([front_page, back_page])

formats = {'A4': A4, 'letter': letter}
card_width = 2.75 * inch
card_height = 3.75 * inch

for page_format in formats:
    canvas = Canvas(os.path.join(output_path, '{}.pdf'.format(page_format)),
                    pagesize=landscape(formats[page_format]))
    width, height = landscape(formats[page_format])
    width_margin = (width - 3 * card_width) / 2
    height_margin = (height - 2 * card_height) / 2
    for page in pages:
        for i in range(len(page)):
            if page[i]:
                width_pos = (width_margin + i * card_width
                             if i < cards_on_page / 2
                             else width_margin + (i - cards_on_page / 2) * card_width)
                height_pos = height_margin + card_height if i < cards_on_page / 2 else height_margin
                canvas.drawImage(page[i], width_pos, height_pos,
                                 card_width, card_height, anchor='sw')

        canvas.showPage()

    canvas.save()

In [116]:
# Generate MakePlayingCards outputs
input_path = os.path.join(IMAGES_EONS_PATH, 'png800BleedMPC', SET_UUID)
output_path = os.path.join(OUTPUT_MPC_PATH, set_name)
create_folder(output_path)

#with py7zr.SevenZipFile(os.path.join(output_path, '{}.7z'.format(set_name)), 'w') as zip_obj:
with zipfile.ZipFile(os.path.join(output_path, '{}.zip'.format(set_name)), 'w') as zip_obj:
    for _, _, filenames in os.walk(input_path):
        for filename in filenames:
            parts = filename.split('-')
            if parts[-1] != '1.png':
                continue

            back_filepath = os.path.join(input_path, '{}-2.png'.format('-'.join(parts[:-1])))
            if not os.path.exists(back_filepath):
                if parts[2] == 'p':
                    back_filepath = os.path.join(IMAGES_BACK_PATH, 'playerBackUnofficialMPC.png')
                elif parts[2] == 'e':
                    back_filepath = os.path.join(IMAGES_BACK_PATH, 'encounterBackUnofficialMPC.png')
                else:
                    print('Missing card back for {}, removing the file from MakePlayingCards archive'
                          .format(filename))
                    continue

            if parts[1] == 'p':
                for i in range(3):
                    parts[1] = str(i + 1)
                    front_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-1\.png)', '',
                                                          'front/{}'.format('-'.join(parts)))))
                    back_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-2\.png)', '',
                                                         'back/{}'.format('{}-2.png'.format('-'.join(parts[:-1]))))))
                    zip_obj.write(os.path.join(input_path, filename), front_zippath)
                    zip_obj.write(back_filepath, back_zippath)
            else:
                front_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-1\.png)', '',
                                                      'front/{}'.format('-'.join(parts)))))
                back_zippath = re.sub('-(?:e|p)-', '-', re.sub('-+', '-', re.sub('.{36}(?=-2\.png)', '',
                                                     'back/{}'.format('{}-2.png'.format('-'.join(parts[:-1]))))))
                zip_obj.write(os.path.join(input_path, filename), front_zippath)
                zip_obj.write(back_filepath, back_zippath)

        break

Now there should be the following outputs:

1. `Output/DB/<set name>/`: 300 dpi JPG images for general purposes.
2. `Output/MakePlayingCards/<set name>`: archive of 800 dpi PNG images
    to be printed on MakePlayingCards.com.
3. `Output/OCTGN/<octgnid>/:` `set.xml` and `o8c` image pack (300 dpi JPG) for OCTGN.
    Add the latter using the "Add Image Packs" button from within OCTGN.
4. `Output/PDF/<set name>/`: PDF files in `A4` and `letter` format for home printing.

In [ ]:
# Copy set folder to OCTGN directory
#from distutils.dir_util import copy_tree

#dirOCTGN = os.path.expanduser('~') + '\\Documents\\OCTGN\\'
#if not os.path.isdir(dirOCTGN):
#    print('Invalid path!')

#setspath = dirOCTGN + '\\GameDatabase\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\'

#if os.path.isdir(dirOCTGN):
#    if os.path.isdir(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)):
#        copy_tree(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID), os.path.join(setspath, SET_UUID))
#        print('Copied successfully.')
#    else:
#        print('No folder found to copy.')
#else:
#    print('OCTGN Sets folder not found.')